# О коллекциях в `python`

## Разнообразие коллекций

В `python` в любой момент времени имеется доступ к нескольким видам коллекций, среди которых:
- [list](https://docs.python.org/3/library/stdtypes.html#lists) --- списки;
- [tuple](https://docs.python.org/3/library/stdtypes.html#tuples) --- кортежи;
- [range](https://docs.python.org/3/library/stdtypes.html#ranges) --- диапазоны;
- [str](https://docs.python.org/3/library/stdtypes.html#str) --- строки;
- [bytearray](https://docs.python.org/3/library/stdtypes.html#bytearray) и [bytes](https://docs.python.org/3/library/stdtypes.html#bytes) --- изменяемый и неизменяемый  массивы байтов;
- [set](https://docs.python.org/3/library/stdtypes.html#set) и [frozenset](https://docs.python.org/3/library/stdtypes.html#frozenset) --- изменяемые и неизменяемые множества;
- [dict](https://docs.python.org/3/library/stdtypes.html#dict) --- словари (ассоциативные массивы, хеш таблицы).

Ещё больше коллекций доступно, если брать в расчет модули стандартной библиотеки:
- [array.array](https://docs.python.org/3/library/array.html) --- массивы численных значений;
- [collections.deque](https://docs.python.org/3/library/collections.html#collections.deque) --- двухсторонняя очередь;
- [collections.OrderedDict](https://docs.python.org/3/library/collections.html#collections.OrderedDict) --- упорядоченный словарь;
- [collections.defaultdict](https://docs.python.org/3/library/collections.html#collections.defaultdict) --- словарь, со значениями по-умолчанию;
- и т.д.

А ещё ведь существуют разнообразные коллекции из сторонних библиотек. Может показаться, что познакомиться со всеми коллекциями в `python` --- непосильная задача. Однако, можно значительно упростить себе задачу, если выявить между некоторыми коллекциями сходства. Благо сам `python` выделяет среди коллекций некоторую иерархию, которая располагает все свои встроенные коллекции и многие сторонние по полочкам. 

## Базовые понятия

Для начала введём три самых базовых понятия с точки зрения `python`.
1. Объект считается **контейнером** (**`Container`**), если у него можно спросить, содержит ли он какой-то произвольный элемент `x` (**does it `contain` x?** ). Более формально, если `A` контейнер, то для любого `x` выражение `x in A` должно обрабатываться без ошибок.
2. Объект считается **итерируемым** (**`Iterable`**), если по нему можно пробежаться в цикле (**`iterate` over**). Т.е. `A` --- итерируемый объект, если можно написать `for x in A: ...`. 
3. Объект считается **объектом ограниченной длины** (**`Sized`**), если у него можно спросить количество элементов (длина, размер, **`size`**) методом [len](https://docs.python.org/3/library/functions.html#len). Т.е. `A` --- объект ограниченной длинны, если `len(A)` вычисляется без ошибок и возвращает целое неотрицательное число.  

Три приведенных выше определения задают три класса объектов, которые хоть и пересекаются между собой, но не совпадают полностью. 


### Пример: контейнер, но не итерируемый и без длины

Представим себе объект, который будет представлять из себя множество точек внутри интервала $(a, b)$. Ниже приводится реализация класса `Interval`, который позволяет проверять наличие произвольного действительного числа внутри соответствующего интервала ключевым словом `in`. 

```{note}
Не обращайте внимания на детали реализации класса `Interval`: о создании пользовательских классов речь пойдет значительно позже. Пока достаточно обратить внимание на то, как в коде происходит обращение с экземпляром данного класса.
```

In [1]:
from dataclasses import dataclass

@dataclass
class Interval:
    a: float
    b: float

    def __contains__(self, element: float) -> bool:
        return self.a < element < self.b

interval = Interval(0, 1)
for x in (-0.5, 0.5, 1.5):
    contains = x in interval
    print(f"{x=}: {contains=}")


x=-0.5: contains=False
x=0.5: contains=True
x=1.5: contains=False


Теперь зададимся вопросам, можно ли в каком-то разумном порядке пробежаться по числам из интервала $(a, b)$? Ответ на данный вопрос --- нет. Мало того, что внутри отрезка $(a, b)$ находится бесконечное множество чисел, так это множество ещё и не является [счетным](https://ru.wikipedia.org/wiki/%D0%A1%D1%87%D1%91%D1%82%D0%BD%D0%BE%D0%B5_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2%D0%BE) (более конкретно, оно является [континуумом](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D1%82%D0%B8%D0%BD%D1%83%D1%83%D0%BC_(%D1%82%D0%B5%D0%BE%D1%80%D0%B8%D1%8F_%D0%BC%D0%BD%D0%BE%D0%B6%D0%B5%D1%81%D1%82%D0%B2))), что как раз и значит, что даже теоретически невозможно поставить в соответствие каждому числу из интервала $(a, b)$ уникальное натуральное число (его порядковый номер).

Итого, такой объект имеет все полномочия называться контейнером, но он не удовлетворяет определениям итерируемого объекта и объекта ограниченной длины.

### Пример: итерируемый, но не контейнер и не имеет длины

Забегая вперед, можно привести пример объекта, по которому можно пробежаться в цикле, но который не является контейнером и не имеет определенного размера. В `python` есть специальный тип **генераторов**, для создания которых даже предусмотрен свой специальный синтаксис (см. [](generators)). 

Генераторы очень похожи на итераторы, но в отличии от них по-требованию выдают не существующие элементы какого-то итерируемого объекта, а вычисляют новые значения по какому-то правилу. В зависимости от того, какое правило использовалось при создании генератора, последовательность элементов, генерируемая им, может исчерпаться за конечное число шагов, а может длится бесконечно. При этом в общем случае невозможно автоматически заранее вычислить длину этой последовательности. 

Поэтому генераторы не поддерживают вызов функции `len`, а значит не являются объектами ограниченной длины. Проверить наличие заданного элемента в последовательности, генерируемой заданным генератором, тоже невозможно заблаговременно. Поэтому и контейнером генератор не является. 

## Коллекции 

**`Коллекция`** --- объект, который одновременно является **контейнером**, **итерируемым объектом** и **объектом ограниченный длинны**. 
```{figure} /_static/lecture_specific/types/collections_venn.svg
```
С бытовой точки зрения, коллекция --- нечто, хранящее в себе совокупность объектов. 
- Эта совокупность не может быть бесконечной, т.к. она физически хранится внутри коллекции, а значит должна быть ограниченного размера, т.е. допускает вызов функции `len`.
- Т.к. эта совокупность ограничена, то мы всегда можем пробежаться по ней (например, в случайном порядке) в цикле, а значит она является итерируемым объектом.
- Т.к. мы всегда можем пробежаться по её элементам, то мы всегда можем проверить наличия произвольного элемента в коллекции (например, просто сравнив каждый элемент коллекции на равенство в цикле), т.е. коллекция является контейнером. 

Разработчики языка `python` смотрят на эту иерархию понятий, как на иерархию классов, где абстрактный класс `collection` наследуется от всех трех выше обозначенных абстрактных классов.
```{figure} /_static/lecture_specific/types/collections_hierarchy.svg
```

```{note}
Более конкретно, каждый абстрактный класс в иерархии задаёт интерфейс: только объекты, удовлетворяющие заданному интерфейсу, можно считать экземплярами этого класса. Все производные классы тоже обязанны удовлетворять этому интерфейсу, а значит экземпляры класса `Collection` должны одновременно удовлетворять интерфейсам классов `Container`, `Iterable` и `Sized`.
```

## Дальнейшее ветвление в иерархии

Выше было перечисленно сравнительно больше количество объектов, являющихся коллекциями. Дальнейшая их классификация обычно производится по способу получения доступа к элементам. По такому принципу можно выделить два наиболее широких класса --- последовательности (`Sequence`) и отображения (`Mapping`), хотя ещё существуют множества (`Set`) и другие. 

```{figure} /_static/lecture_specific/types/sequence_vs_mapping.svg
```

Про последовательности речь пойдет на следующей странице, а с единственным представителем класса `Mapping` мы познакомимся несколько позже (см. [](dictionaries)).

## Операции над коллекциями

Такое выделение иерархии типов в `python` не случайно. Надо объектами каждого класса можно осуществлять определенный набор операций. Это позволяет применять эти операции в том числе и экземпляров всех производных классов. Ниже приводится два примера таких операций, однако, это далеко не исчерпывающий список: в документации `python` слово `iterable` встречается повсеместно, т.к. возможности пробежаться по элементам в цикле достаточно для очень широкого круга операций. 

### Проверка на принадлежность

У любого контейнера можно проверять наличие элемента, т.е. писать выражение следующего вида. 
```python
x in container
```
Т.к. все коллекции по совместительству являются контейнерами, то гарантируется корректность и следующего выражения.
```python
x in collection
```

In [2]:
t = ("a", "b", "c")
l = ["a", "b", "c"]
s = "abc"

x = "a"
for collection in (s, l, t):
    print(x in collection)

True
True
True


### Операции порядка

Если все элементы итерируемого объекта однородны и их можно сравнить на предмет "больше/меньше" (написать, `x < y`), то их можно ранжировать: искать минимальное/максимальное значение и сортировать. Для этого используются встроенные функции [min](https://docs.python.org/3/library/functions.html#min), [max](https://docs.python.org/3/library/functions.html#max) и [sorted](https://docs.python.org/3/library/functions.html#sorted).
```python
m = min(iterable)
M = max(iterable)
s = sorted(iterable)
```
Т.к. коллекции являются по совместительству и итерируемыми объектами, то среди них тоже можно искать минимум, максимум, и их тоже можно сортировать (всё это при условии однородности хранимых элементов).

